In [25]:
!pip install wget
!pip install anndata

# Donwloading the data
1) Begin by running the download_data.py in the SingleCellProcess folder (please run the above code if you do not have wget installed.
2) There are 6 datasets that will be downloaded
3) GSE75748 cell consists of different cells originating from H1 stem cells
4) GSE75748 time consists of H1 cells sequenced in a hypoxic environment at different time. The goal is to see changes in gene expression.
5) The last dataset consists of 4 sub-dataset. GSE84133 human 1,2,3,4. I would like you all to merge the dataset together to generate a large dataset of about 9000 cells.
6) Once the dataset is downloaded, the data can be found as a csv file (open with excel) under ./SingleCellDataProcess/data/

# Task for this week
1) Using the above datasets, construct a machine learning classification model to achieve the best results.
2) Begin by preprocessing the data using the scanpy tutorial (see code below to generate the AnnData type). 

# Some tips
1) Run a basic statistics on the gene expression, as shown in scanpy, and decide how much data to keep.
2) Look at the class size of each cell types. Unbalanced data is extremely hard to train. For example, if I divide the data into a training and testing data with 100 bunnies and 10 turtles, both training and testing data will have alot more bunnies. Training your model with such data can show bias towards bunnies. You may need to balance the data. 
3) When you traing any models, fix a random_state. All machine learning models utilize stochastic gradient descent, which computes derivatives using only parts of your data. How these data are chosen is based on random number generator. By fixing the random_state, you will always have the same result. To make sure your method is working properly, test your method with multiple (about 5) random state. For actual papers, many people do 30-100 random_state. Check [this video](https://youtu.be/vMh0zPT0tLI?si=r7mbTQSdTnU4qmCe) for more info on stocahstic gradient descent.
3) For any questions (small or big) please let me know

# Models to test
There are hundreds of classification models, each with their own upside and downside.
The model you select will depend on the number of data you have, the number of features, underlying assumptions, and more. Just because you have a lot of features, it does not mean you have a lot of 'good' features. ScRNA-seq is a fantastic example of bad large number of features. Additionally, if your features are in fact linearly dependent, then your features may also be bad.

## You can check out the following to test your model (I am only listing ones that you can implement easily using the scikit-learn package from python)
1) [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)
2) [Linear Support Vector](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC)
3) [Kernel Support Vector](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)
- look at the 'kernel' and 'degree' parameters. These values can be used to optimize your model
4) [Nearest Neighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)
5) [Decission Tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
6) [Random Forest Decission Tree](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)
7) [Gradient Boosting Decision Tree](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier)
8) [Artificial Neural Network](https://www.tensorflow.org/tutorials/quickstart/beginner)
9) [Recurrent Neural Network](https://www.tensorflow.org/guide/keras/working_with_rnns)

There is alot of algorithm I have mentioned. You don NOT have to test all of this (this would take you multiple weeks to get it done). What is really important here is the hands-on experience and practicing reading the documentation. There are lots and lots of resources online on what these algorithms are. I would watch statquest's youtube video on what these algorithms are. There are a lot of theoretical concepts with these algorithm as well if you are interested.

In general all the methods (aside from ANN and RNN) uses the fit() and predict() workflow. Play around with it

# Cross validation
- In machine learning, we usually implement cross-validation to make sure our method is working properly. 
- For example, in a 5 fold cross validation, you divide the data into 5 parts, and use 4 of them to train the model, and 1 to test the model. We use the data we tested to compute the accuracy of the model.
- We can do this using scikit-learn's [Kfold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)
- See the code below for an example. I use support vector machine as an example

In [23]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X = np.random.rand(100,3)   #just some random matrix
y = np.random.choice(5, size = 100)  # some random class

kf = KFold(n_splits=5, shuffle = True, random_state = 1)  #change this value if you are doing more than 5. For this week, just do 5

for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train = X[train_index, :]; y_train = y[train_index]
    X_test = X[test_index, :]; y_test = y[test_index]
    
    mySVC = SVC(kernel = 'rbf', degree = 5 ,random_state = 1)  # you should fix random_state = 1, and just change the kfold shuffling
    mySVC.fit(X_train, y_train)
    y_pred = mySVC.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('Fold', i, 'Accuracy:', score)

Fold 0 Accuracy: 0.15
Fold 1 Accuracy: 0.3
Fold 2 Accuracy: 0.1
Fold 3 Accuracy: 0.1
Fold 4 Accuracy: 0.3


- Notice that for kfold, I use the 'shuffle = Ture' and 'random_state = 1'. I fix the random state so that the data shuffling and the splitting is always the same. You should test different random_state values to see if your model achieves similar result

# Data loading

Scanpy is built on [AnnData](https://anndata.readthedocs.io/en/latest/). I will show how to construct the AnnData

In [25]:
import anndata as ad
import pandas as pd
import numpy as np

In [5]:
data = 'GSE75748cell'
record_csv = pd.read_csv('./SingleCellDataProcess/data/' + data + '/' +  data + '_data.csv')
labels_csv = pd.read_csv('./SingleCellDataProcess/data/' + data + '/' + data + '_labels.csv')

In [14]:
gene_list = list(record_csv['Unnamed: 0'])
sample_name = list(record_csv.keys())[1:]
counts = record_csv.values[:, 1:].astype(float)
print('First 10 genes', gene_list[:10])
print('Data size:', counts.shape)

First 10 genes ['MKL2', 'CD109', 'ABTB1', 'MAST2', 'KAT5', 'WWC2', 'CD163', 'MYL2', 'UBE2Z', 'RGPD4']
Data size: (19097, 1018)


In [17]:
adata = ad.AnnData(counts.T)
adata.obs_names = sample_name
adata.var_names = gene_list
counts = None; sample_name = None; gene_list = None;

You can now use scanpy.

##  Merging GSE84133 data

In [38]:
data_vec = ['GSE84133human1', 'GSE84133human2', 'GSE84133human3', 'GSE84133human4']
sample_name_list = []
counts_vec = []
for data in data_vec:
    record_csv = pd.read_csv('./SingleCellDataProcess/data/' + data + '/' +  data + '_data.csv')
    labels_csv = pd.read_csv('./SingleCellDataProcess/data/' + data + '/' + data + '_labels.csv')
    gene_list = list(record_csv['Unnamed: 0'])
    sample_name = list(record_csv.keys())[1:]
    sample_name_list += sample_name
    counts = record_csv.values[:, 1:].astype(float)
    counts_vec.append(counts)
counts_vec = np.concatenate(counts_vec, axis = 1)
adata = ad.AnnData(counts_vec.T)
adata.obs_names = sample_name_list
adata.var_names = gene_list
counts_vec = None; sample_name_list = None; gene_list = None;

# Extra credit (just for discussion)
- When we compute the accuracy, it is extremely important to think about what the result means?
- For example, which cell types is misclassified the most? These can be used to further fine tune the model, or show something interesting about the data
- For GSE84133 large data, you should test your model with one of the data, such as GSE84133human3

# Some benchmark
- GSE75748cell: about 0.8 accuracy
- GSE75748time: 0.74-0.77
- GSE84133: this really depends on what method you use. It can be VERY good with deep learning.